In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import jaccard_score
import pandas as pd
import random

2025-05-22 06:03:04.450238: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-22 06:03:04.487341: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

In [3]:
# Define paths
BASE_PATH = "../data"
FOLDS = ["Fold1", "Fold2", "Fold3", "Fold4", "Fold5"]
IMG_SIZE = (256, 256)
PIXELS_PER_CM = 72
BATCH_SIZE = 4
EPOCHS = 50
LEARNING_RATE = 1e-4

OUTPUT_DIR = "Unet_result"
FIGURE_DIR = os.path.join(OUTPUT_DIR, "figure")
MODELS_DIR = os.path.join(OUTPUT_DIR, "models")
RESULTS_DIR = os.path.join(OUTPUT_DIR, "results")

# Create output directories
for dir_path in [OUTPUT_DIR, FIGURE_DIR, MODELS_DIR, RESULTS_DIR]:
    os.makedirs(dir_path, exist_ok=True)

In [4]:
def load_image(image_path):
    """載入並前處理圖像"""
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"無法讀取圖像: {image_path}")
        return np.zeros((IMG_SIZE[0], IMG_SIZE[1], 1), dtype=np.float32)

    img = cv2.resize(img, IMG_SIZE)
    img = img.astype(np.float32) / 255.0
    img = np.expand_dims(img, axis=-1)  # (H, W, 1)
    return img

In [5]:
def load_mask(mask_path):
    """載入並前處理遮罩，確保嚴格的二值化"""
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if mask is None:
        print(f"無法讀取遮罩: {mask_path}")
        return np.zeros((IMG_SIZE[0], IMG_SIZE[1], 1), dtype=np.float32)

    mask = cv2.resize(mask, IMG_SIZE)
    mask = (mask > 127).astype(np.float32)
    mask = np.expand_dims(mask, axis=-1)
    return mask

In [6]:
def augment_data(image, mask):
    """對圖像和遮罩進行數據增強"""
    # 確保輸入形狀正確
    if image.ndim != 3 or image.shape[2] != 1:
        image = cv2.resize(image, IMG_SIZE)
        if image.ndim == 2:
            image = np.expand_dims(image, axis=-1)
        elif image.shape[2] > 1:
            image = np.expand_dims(image[:, :, 0], axis=-1)

    if mask.ndim != 3 or mask.shape[2] != 1:
        mask = cv2.resize(mask, IMG_SIZE)
        if mask.ndim == 2:
            mask = np.expand_dims(mask, axis=-1)
        elif mask.shape[2] > 1:
            mask = np.expand_dims(mask[:, :, 0], axis=-1)

    # 暫時將遮罩從 (256,256,1) 轉換為 (256,256) 進行處理
    mask_2d = np.squeeze(mask)

    # 水平翻轉
    if random.random() > 0.5:
        image = np.fliplr(image)
        mask_2d = np.fliplr(mask_2d)

    # 旋轉±10度
    if random.random() > 0.5:
        angle = random.uniform(-10, 10)
        h, w = image.shape[:2]
        center = (w // 2, h // 2)
        rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)

        image = cv2.warpAffine(image, rotation_matrix, (w, h),
                               flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
        mask_2d = cv2.warpAffine(mask_2d, rotation_matrix, (w, h),
                                 flags=cv2.INTER_NEAREST, borderMode=cv2.BORDER_CONSTANT)

    # 確保返回的形狀為 (256,256,1)
    image = np.reshape(image, (IMG_SIZE[0], IMG_SIZE[1], 1))
    mask = np.reshape(mask_2d, (IMG_SIZE[0], IMG_SIZE[1], 1))

    image = image.astype(np.float32)
    mask = (mask > 0.5).astype(np.float32)

    return image, mask

In [7]:
def match_images_and_masks(image_paths, mask_paths):
    """匹配圖像和遮罩路徑，對大小寫不敏感"""
    image_basenames = {}
    for p in image_paths:
        basename = os.path.splitext(os.path.basename(p))[0].lower()
        image_basenames[basename] = p

    mask_basenames_map = {}
    for p in mask_paths:
        basename = os.path.splitext(os.path.basename(p))[0].lower()
        if basename.startswith('mask_'):
            mask_basenames_map[basename[5:]] = p
        else:
            mask_basenames_map[basename] = p

    matched_image_paths = []
    matched_mask_paths = []
    unmatched_images = []

    for img_key, img_path in image_basenames.items():
        if img_key in mask_basenames_map:
            matched_image_paths.append(img_path)
            matched_mask_paths.append(mask_basenames_map[img_key])
        else:
            unmatched_images.append(img_path)

    if unmatched_images:
        print(f"未匹配的圖像檔案: {unmatched_images[:5]} (總共 {len(unmatched_images)} 個)")

    print(f"匹配到 {len(matched_image_paths)} 對圖像和遮罩 (原始圖像數: {len(image_paths)}, 原始遮罩數: {len(mask_paths)})")
    return matched_image_paths, matched_mask_paths

In [8]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, image_paths, mask_paths, batch_size=BATCH_SIZE, augment=False, shuffle=True):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.batch_size = batch_size
        self.augment = augment
        self.shuffle = shuffle
        self.indexes = np.arange(len(image_paths))
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        batch_indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        current_batch_size = len(batch_indexes)

        batch_images = np.zeros((current_batch_size, IMG_SIZE[0], IMG_SIZE[1], 1), dtype=np.float32)
        batch_masks = np.zeros((current_batch_size, IMG_SIZE[0], IMG_SIZE[1], 1), dtype=np.float32)

        for i, idx in enumerate(batch_indexes):
            try:
                image = load_image(self.image_paths[idx])
                mask = load_mask(self.mask_paths[idx])

                if self.augment:
                    image, mask = augment_data(image, mask)

                # 確保形狀正確
                image = np.reshape(image, (IMG_SIZE[0], IMG_SIZE[1], 1))
                mask = np.reshape(mask, (IMG_SIZE[0], IMG_SIZE[1], 1))

                batch_images[i] = image
                batch_masks[i] = mask
            except Exception as e:
                print(f"處理圖像 {self.image_paths[idx]} 或遮罩 {self.mask_paths[idx]} 時出錯: {e}")
                batch_images[i] = np.zeros((IMG_SIZE[0], IMG_SIZE[1], 1), dtype=np.float32)
                batch_masks[i] = np.zeros((IMG_SIZE[0], IMG_SIZE[1], 1), dtype=np.float32)
        return batch_images, batch_masks

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

In [9]:
def find_ett_endpoint_with_x(mask_squeezed):
    """尋找氣管內管的端點 (y座標) 和對應的 x 座標中點"""
    binary_mask = (mask_squeezed > 0.5).astype(np.uint8)
    white_pixels_y, white_pixels_x = np.where(binary_mask > 0)

    if len(white_pixels_y) == 0:
        return 0, 0

    endpoint_y = np.max(white_pixels_y)
    x_at_endpoint_y = white_pixels_x[white_pixels_y == endpoint_y]
    if len(x_at_endpoint_y) == 0:
        endpoint_x = np.mean(white_pixels_x) if len(white_pixels_x) > 0 else 0
    else:
        endpoint_x = np.mean(x_at_endpoint_y)

    return int(round(endpoint_y)), int(round(endpoint_x))

In [10]:
def calculate_custom_metrics(gt_endpoints_y, pred_endpoints_y, pixels_per_cm=PIXELS_PER_CM):
    """計算自定義評估指標 (僅基於 y 座標)"""
    gt_endpoints_y = np.array(gt_endpoints_y)
    pred_endpoints_y = np.array(pred_endpoints_y)

    if len(gt_endpoints_y) == 0 or len(pred_endpoints_y) == 0:
        print("警告: 端點列表為空，無法計算自定義指標。")
        return {
            'mean_error_cm': np.nan,
            'acc_within_05cm': np.nan,
            'acc_within_10cm': np.nan
        }
    if len(gt_endpoints_y) != len(pred_endpoints_y):
        print(f"警告: gt_endpoints ({len(gt_endpoints_y)}) 和 pred_endpoints ({len(pred_endpoints_y)}) 長度不匹配。")
        min_len = min(len(gt_endpoints_y), len(pred_endpoints_y))
        gt_endpoints_y = gt_endpoints_y[:min_len]
        pred_endpoints_y = pred_endpoints_y[:min_len]
        if min_len == 0:
            return {
                'mean_error_cm': np.nan,
                'acc_within_05cm': np.nan,
                'acc_within_10cm': np.nan
            }

    abs_errors_px = np.abs(pred_endpoints_y - gt_endpoints_y)
    abs_errors_cm = abs_errors_px / pixels_per_cm

    mean_error_cm = np.mean(abs_errors_cm)
    accuracy_05cm = 100 * np.mean(abs_errors_px <= (pixels_per_cm / 2))
    accuracy_10cm = 100 * np.mean(abs_errors_px <= pixels_per_cm)

    return {
        'mean_error_cm': mean_error_cm,
        'acc_within_05cm': accuracy_05cm,
        'acc_within_10cm': accuracy_10cm
    }

In [11]:
def iou_metric(y_true, y_pred):
    if y_true.ndim > 2:
        y_true = np.squeeze(y_true)
    if y_pred.ndim > 2:
        y_pred = np.squeeze(y_pred)

    y_true_flat = y_true.flatten()
    y_pred_flat = (y_pred > 0.5).flatten().astype(np.uint8)
    if np.sum(y_true_flat) == 0 and np.sum(y_pred_flat) == 0:
        return 1.0
    return jaccard_score(y_true_flat, y_pred_flat, zero_division=1)

In [12]:
def post_process_prediction(pred):
    if pred.ndim == 2:
        pred = pred[..., np.newaxis]
    elif pred.ndim == 4:
        pred = pred[0]

    pred_binary = (pred > 0.5).astype(np.uint8)

    kernel = np.ones((3, 3), np.uint8)
    pred_binary = cv2.erode(pred_binary, kernel, iterations=1)
    pred_binary = cv2.dilate(pred_binary, kernel, iterations=1)

    if pred_binary.ndim == 2:
        pred_binary = pred_binary[..., np.newaxis]

    return pred_binary

In [13]:
def unet_model(input_shape=(256, 256, 1)):
    inputs = layers.Input(input_shape)

    c1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    c1 = layers.BatchNormalization()(c1)
    c1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(c1)
    c1 = layers.BatchNormalization()(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)
    skip_connections = [c1]

    c2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(p1)
    c2 = layers.BatchNormalization()(c2)
    c2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(c2)
    c2 = layers.BatchNormalization()(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)
    skip_connections.append(c2)

    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(p2)
    c3 = layers.BatchNormalization()(c3)
    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(c3)
    c3 = layers.BatchNormalization()(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)
    skip_connections.append(c3)

    c4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(p3)
    c4 = layers.BatchNormalization()(c4)
    c4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(c4)
    c4 = layers.BatchNormalization()(c4)
    p4 = layers.MaxPooling2D((2, 2))(c4)
    skip_connections.append(c4)

    c5 = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(p4)
    c5 = layers.BatchNormalization()(c5)
    c5 = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(c5)
    c5 = layers.BatchNormalization()(c5)

    u6 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, skip_connections.pop()])
    c6 = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(u6)
    c6 = layers.BatchNormalization()(c6)
    c6 = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(c6)
    c6 = layers.BatchNormalization()(c6)

    u7 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, skip_connections.pop()])
    c7 = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(u7)
    c7 = layers.BatchNormalization()(c7)
    c7 = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(c7)
    c7 = layers.BatchNormalization()(c7)

    u8 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = layers.concatenate([u8, skip_connections.pop()])
    c8 = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(u8)
    c8 = layers.BatchNormalization()(c8)
    c8 = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(c8)
    c8 = layers.BatchNormalization()(c8)

    u9 = layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = layers.concatenate([u9, skip_connections.pop()])
    c9 = layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(u9)
    c9 = layers.BatchNormalization()(c9)
    c9 = layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(c9)
    c9 = layers.BatchNormalization()(c9)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid', kernel_initializer='he_normal')(c9)

    model = models.Model(inputs, outputs)
    return model

In [ ]:
# def post_process_prediction(pred):
#     if pred.ndim == 2:
#         pred = pred[..., np.newaxis]
#     elif pred.ndim == 4:
#         pred = pred[0]

#     pred_binary = (pred > 0.5).astype(np.uint8)

#     kernel = np.ones((3, 3), np.uint8)
#     pred_binary = cv2.erode(pred_binary, kernel, iterations=1)
#     pred_binary = cv2.dilate(pred_binary, kernel, iterations=1)

#     if pred_binary.ndim == 2:
#         pred_binary = pred_binary[..., np.newaxis]

#     return pred_binary

In [15]:
def dice_loss(y_true, y_pred):
    smooth = 1e-6
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return 1 - ((2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth))

In [16]:
def iou_score(y_true, y_pred):
    y_true = tf.cast(tf.greater(y_true, 0.5), tf.float32)
    y_pred = tf.cast(tf.greater(y_pred, 0.5), tf.float32)

    intersection = tf.reduce_sum(y_true * y_pred, axis=[1, 2, 3])
    union = tf.reduce_sum(y_true, axis=[1, 2, 3]) + tf.reduce_sum(y_pred, axis=[1, 2, 3]) - intersection

    iou = (intersection + 1e-10) / (union + 1e-10)
    return tf.reduce_mean(iou)

In [17]:
def visualize_improved_results(image, true_mask, pred_mask, save_path=None, model_name="U-Net"):
    plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'Microsoft YaHei', 'WenQuanYi Micro Hei']
    plt.rcParams['axes.unicode_minus'] = False

    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    if image.ndim > 2:
        image = np.squeeze(image)
    if true_mask.ndim > 2:
        true_mask_squeezed = np.squeeze(true_mask)
    else:
        true_mask_squeezed = true_mask
    if pred_mask.ndim > 2:
        pred_mask_squeezed = np.squeeze(pred_mask)
    else:
        pred_mask_squeezed = pred_mask

    axes[0].set_title('Image')
    axes[0].imshow(image, cmap='gray')
    axes[0].axis('off')

    axes[1].set_title('Ground Truth Mask')
    background_color_gt = [75/255, 0/255, 130/255]
    bg_gt = np.ones((IMG_SIZE[0], IMG_SIZE[1], 3)) * background_color_gt
    axes[1].imshow(bg_gt)

    gt_display_mask = np.zeros((*true_mask_squeezed.shape, 4))
    gt_display_mask[true_mask_squeezed > 0.5, 1] = 0.8
    gt_display_mask[true_mask_squeezed > 0.5, 3] = 0.9
    axes[1].imshow(gt_display_mask)

    true_endpoint_y, true_endpoint_x = find_ett_endpoint_with_x(true_mask_squeezed)
    if true_endpoint_y > 0:
        axes[1].plot(true_endpoint_x, true_endpoint_y, 'ro', markersize=8, label='GT Endpoint')
        axes[1].text(true_endpoint_x + 5, true_endpoint_y, 'G', color='red', fontsize=12)
    axes[1].axis('off')

    axes[2].set_title('Predicted Mask')
    bg_pred = np.ones((IMG_SIZE[0], IMG_SIZE[1], 3)) * background_color_gt
    axes[2].imshow(bg_pred)

    binary_pred_squeezed = (pred_mask_squeezed > 0.5).astype(np.float32)

    if np.max(binary_pred_squeezed) > 0:
        pred_display_mask = np.zeros((*binary_pred_squeezed.shape, 4))
        pred_display_mask[binary_pred_squeezed > 0, 0] = 0.95
        pred_display_mask[binary_pred_squeezed > 0, 1] = 0.95
        pred_display_mask[binary_pred_squeezed > 0, 3] = 0.9
        axes[2].imshow(pred_display_mask)

        pred_endpoint_y, pred_endpoint_x = find_ett_endpoint_with_x(binary_pred_squeezed)
        if pred_endpoint_y > 0:
            axes[2].plot(pred_endpoint_x, pred_endpoint_y, 'ro', markersize=8, label='Pred Endpoint')
            axes[2].text(pred_endpoint_x + 5, pred_endpoint_y, 'Y', color='red', fontsize=12)
    else:
        axes[2].imshow(pred_mask_squeezed, cmap='viridis_r', alpha=0.7, vmin=0, vmax=1)
        cbar = plt.colorbar(axes[2].images[0], ax=axes[2], fraction=0.046, pad=0.04)
        cbar.set_label('Prediction Confidence')

    axes[2].axis('off')

    plt.suptitle(f"{model_name} Segmentation Results", fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.96])

    if save_path:
        plt.savefig(save_path, bbox_inches='tight', dpi=300)
        plt.close(fig)
    else:
        plt.show()

In [18]:
def train_and_evaluate(fold, paths):
    print(f"=== 訓練 {fold} ===")

    train_img_paths = [os.path.join(paths['train'], f) for f in os.listdir(paths['train']) if f.endswith(('.jpg', '.jpeg'))]
    train_mask_paths = [os.path.join(paths['trainannot'], f) for f in os.listdir(paths['trainannot']) if f.endswith('.png')]

    val_img_paths = [os.path.join(paths['val'], f) for f in os.listdir(paths['val']) if f.endswith(('.jpg', '.jpeg'))]
    val_mask_paths = [os.path.join(paths['valannot'], f) for f in os.listdir(paths['valannot']) if f.endswith('.png')]

    test_img_paths = [os.path.join(paths['test'], f) for f in os.listdir(paths['test']) if f.endswith(('.jpg', '.jpeg'))]
    test_mask_paths = [os.path.join(paths['testannot'], f) for f in os.listdir(paths['testannot']) if f.endswith('.png')]

    print(f"訓練資料: 找到 {len(train_img_paths)} 張圖像, {len(train_mask_paths)} 張遮罩")
    print(f"驗證資料: 找到 {len(val_img_paths)} 張圖像, {len(val_mask_paths)} 張遮罩")
    print(f"測試資料: 找到 {len(test_img_paths)} 張圖像, {len(test_mask_paths)} 張遮罩")

    train_img_paths, train_mask_paths = match_images_and_masks(train_img_paths, train_mask_paths)
    val_img_paths, val_mask_paths = match_images_and_masks(val_img_paths, val_mask_paths)
    test_img_paths, test_mask_paths = match_images_and_masks(test_img_paths, test_mask_paths)

    if len(train_img_paths) == 0 or len(val_img_paths) == 0 or len(test_img_paths) == 0:
        print(f"警告: {fold} 中沒有足夠的匹配數據。跳過此 Fold。")
        return {
            'fold': fold, 'mean_iou': 0, 'mean_error_cm': np.nan,
            'acc_within_05cm': np.nan, 'acc_within_10cm': np.nan
        }, None

    train_generator = DataGenerator(train_img_paths, train_mask_paths, augment=True, shuffle=True)
    val_generator = DataGenerator(val_img_paths, val_mask_paths, augment=False, shuffle=False)

    model = unet_model()
    model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss=dice_loss, metrics=['accuracy', iou_score])

    model_save_path = os.path.join(MODELS_DIR, f'best_model_fold{fold}.h5')
    checkpoint = ModelCheckpoint(model_save_path, monitor='val_iou_score', mode='max', save_best_only=True, verbose=1)
    early_stopping = EarlyStopping(monitor='val_iou_score', mode='max', patience=20, verbose=1)

    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=EPOCHS,
        callbacks=[checkpoint, early_stopping],
        verbose=1
    )

    if os.path.exists(model_save_path):
        print(f"載入最佳模型: {model_save_path}")
        model.load_weights(model_save_path)
    else:
        print(f"警告: 找不到最佳模型檔案 {model_save_path}。將使用訓練結束時的模型進行評估。")

    print(f"=== 評估 {fold} ===")
    gt_endpoints_y = []
    pred_endpoints_y = []
    test_ious = []

    if len(test_img_paths) == 0:
        print(f"警告: {fold} 沒有測試數據進行評估。")
        return {
            'fold': fold, 'mean_iou': 0, 'mean_error_cm': np.nan,
            'acc_within_05cm': np.nan, 'acc_within_10cm': np.nan
        }, history

    for i in range(len(test_img_paths)):
        image = load_image(test_img_paths[i])
        true_mask = load_mask(test_mask_paths[i])

        image_batch = np.expand_dims(image, axis=0)
        pred_mask_batch = model.predict(image_batch)
        pred_mask = pred_mask_batch[0]
        pred_mask = post_process_prediction(pred_mask)

        true_mask_squeezed = np.squeeze(true_mask)
        pred_mask_squeezed = np.squeeze(pred_mask)

        true_endpoint_y_val, _ = find_ett_endpoint_with_x(true_mask_squeezed)
        pred_endpoint_y_val, _ = find_ett_endpoint_with_x(pred_mask_squeezed)

        gt_endpoints_y.append(true_endpoint_y_val)
        pred_endpoints_y.append(pred_endpoint_y_val)

        iou = iou_metric(true_mask, pred_mask)
        test_ious.append(iou)

        img_basename = os.path.splitext(os.path.basename(test_img_paths[i]))[0]
        save_path = os.path.join(FIGURE_DIR, f'{fold}_test_img_{img_basename}.png')
        visualize_improved_results(image, true_mask, pred_mask, save_path, model_name="U-Net")

    custom_metrics = calculate_custom_metrics(gt_endpoints_y, pred_endpoints_y, pixels_per_cm=PIXELS_PER_CM)
    mean_iou = np.mean(test_ious) if test_ious else 0

    results = {
        'fold': fold,
        'mean_iou': mean_iou,
        'mean_error_cm': custom_metrics['mean_error_cm'],
        'acc_within_05cm': custom_metrics['acc_within_05cm'],
        'acc_within_10cm': custom_metrics['acc_within_10cm']
    }

    print(f"測試結果 - {fold}:")
    print(f"  平均 IoU: {mean_iou:.4f}")
    print(f"  平均誤差(公分): {custom_metrics.get('mean_error_cm', 'N/A'):.4f}")
    print(f"  誤差在0.5公分內準確率: {custom_metrics.get('acc_within_05cm', 'N/A'):.2f}%")
    print(f"  誤差在1.0公分內準確率: {custom_metrics.get('acc_within_10cm', 'N/A'):.2f}%")

    return results, history

In [19]:
def main():
    # Define paths for each fold
    PATHS = {
        'Fold1': {
            'train': os.path.join(BASE_PATH, 'Fold1', 'train'),
            'trainannot': os.path.join(BASE_PATH, 'Fold1', 'trainannot'),
            'val': os.path.join(BASE_PATH, 'Fold1', 'val'),
            'valannot': os.path.join(BASE_PATH, 'Fold1', 'valannot'),
            'test': os.path.join(BASE_PATH, 'Fold1', 'test'),
            'testannot': os.path.join(BASE_PATH, 'Fold1', 'testannot')
        },
        'Fold2': {
            'train': os.path.join(BASE_PATH, 'Fold2', 'train'),
            'trainannot': os.path.join(BASE_PATH, 'Fold2', 'trainannot'),
            'val': os.path.join(BASE_PATH, 'Fold2', 'val'),
            'valannot': os.path.join(BASE_PATH, 'Fold2', 'valannot'),
            'test': os.path.join(BASE_PATH, 'Fold2', 'test'),
            'testannot': os.path.join(BASE_PATH, 'Fold2', 'testannot')
        },
        'Fold3': {
            'train': os.path.join(BASE_PATH, 'Fold3', 'train'),
            'trainannot': os.path.join(BASE_PATH, 'Fold3', 'trainannot'),
            'val': os.path.join(BASE_PATH, 'Fold3', 'val'),
            'valannot': os.path.join(BASE_PATH, 'Fold3', 'valannot'),
            'test': os.path.join(BASE_PATH, 'Fold3', 'test'),
            'testannot': os.path.join(BASE_PATH, 'Fold3', 'testannot')
        },
        'Fold4': {
            'train': os.path.join(BASE_PATH, 'Fold4', 'train'),
            'trainannot': os.path.join(BASE_PATH, 'Fold4', 'trainannot'),
            'val': os.path.join(BASE_PATH, 'Fold4', 'val'),
            'valannot': os.path.join(BASE_PATH, 'Fold4', 'valannot'),
            'test': os.path.join(BASE_PATH, 'Fold4', 'test'),
            'testannot': os.path.join(BASE_PATH, 'Fold4', 'testannot')
        },
        'Fold5': {
            'train': os.path.join(BASE_PATH, 'Fold5', 'train'),
            'trainannot': os.path.join(BASE_PATH, 'Fold5', 'trainannot'),
            'val': os.path.join(BASE_PATH, 'Fold5', 'val'),
            'valannot': os.path.join(BASE_PATH, 'Fold5', 'valannot'),
            'test': os.path.join(BASE_PATH, 'Fold5', 'test'),
            'testannot': os.path.join(BASE_PATH, 'Fold5', 'testannot')
        }
    }

    all_results = []
    histories = {}

    for fold_name, paths in PATHS.items():
        print(f"\nProcessing {fold_name}...")
        try:
            results, history = train_and_evaluate(fold_name, paths)
            if results:
                all_results.append(results)
            if history:
                histories[fold_name] = history.history
        except Exception as e:
            print(f"處理 {fold_name} 時發生未預期錯誤: {e}")
            import traceback
            traceback.print_exc()
            continue

    if not all_results:
        print("錯誤: 沒有成功處理任何fold。請檢查數據路徑、文件格式和程式碼錯誤。")
        return

    valid_results = [r for r in all_results if r and not any(np.isnan(val) for val in [
        r.get('mean_iou', np.nan), r.get('mean_error_cm', np.nan),
        r.get('acc_within_05cm', np.nan), r.get('acc_within_10cm', np.nan)
    ])]

    if not valid_results:
        print("錯誤: 所有 Fold 的結果都無效，無法計算平均績效。")
        return

    mean_results = {
        'fold': 'Average',
        'mean_iou': np.mean([r['mean_iou'] for r in valid_results if 'mean_iou' in r]),
        'mean_error_cm': np.mean([r['mean_error_cm'] for r in valid_results if 'mean_error_cm' in r]),
        'acc_within_05cm': np.mean([r['acc_within_05cm'] for r in valid_results if 'acc_within_05cm' in r]),
        'acc_within_10cm': np.mean([r['acc_within_10cm'] for r in valid_results if 'acc_within_10cm' in r])
    }

    all_results.append(mean_results)

    results_df = pd.DataFrame(all_results)
    results_path = os.path.join(RESULTS_DIR, 'unet_results.csv')
    results_df.to_csv(results_path, index=False)

    print("\n=== 所有Fold平均績效 ===")
    print(f"平均 IoU: {mean_results['mean_iou']:.4f}")
    print(f"平均誤差(公分): {mean_results['mean_error_cm']:.4f}")
    print(f"誤差在0.5公分內準確率: {mean_results['acc_within_05cm']:.2f}%")
    print(f"誤差在1.0公分內準確率: {mean_results['acc_within_10cm']:.2f}%")

    print(f"\n所有結果已保存至: {results_path}")

    for fold_name, history_data in histories.items():
        if history_data:
            plt.figure(figsize=(12, 4))
            plt.subplot(1, 2, 1)
            plt.plot(history_data['loss'], label='Train Loss')
            if 'val_loss' in history_data:
                plt.plot(history_data['val_loss'], label='Val Loss')
            plt.title(f'{fold_name} - Loss')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.legend()

            plt.subplot(1, 2, 2)
            if 'iou_score' in history_data:
                plt.plot(history_data['iou_score'], label='Train IoU')
            if 'val_iou_score' in history_data:
                plt.plot(history_data['val_iou_score'], label='Val IoU')
            plt.title(f'{fold_name} - IoU Score')
            plt.xlabel('Epoch')
            plt.ylabel('IoU')
            plt.legend()

            plt.tight_layout()
            history_fig_path = os.path.join(FIGURE_DIR, f'{fold_name}_training_history.png')
            plt.savefig(history_fig_path)
            plt.close()
            print(f"{fold_name} 的訓練歷史曲線已保存至: {history_fig_path}")

if __name__ == "__main__":
    main()


Processing Fold1...
=== 訓練 Fold1 ===
訓練資料: 找到 287 張圖像, 287 張遮罩
驗證資料: 找到 47 張圖像, 47 張遮罩
測試資料: 找到 47 張圖像, 47 張遮罩
未匹配的圖像檔案: ['../data/Fold1/train/img_1.2.826.0.1.3680043.8.498.10001065121843652267743449160233082683.jpg', '../data/Fold1/train/img_1.2.826.0.1.3680043.8.498.10001175380298620851477409998730672515.jpg', '../data/Fold1/train/img_1.2.826.0.1.3680043.8.498.10001274045312501651093242392099983211.jpg', '../data/Fold1/train/img_1.2.826.0.1.3680043.8.498.10002446304107330308555550280339793610.jpg', '../data/Fold1/train/img_1.2.826.0.1.3680043.8.498.10003638361010097105432298560780077394.jpg'] (總共 144 個)
匹配到 143 對圖像和遮罩 (原始圖像數: 287, 原始遮罩數: 287)
匹配到 47 對圖像和遮罩 (原始圖像數: 47, 原始遮罩數: 47)
匹配到 47 對圖像和遮罩 (原始圖像數: 47, 原始遮罩數: 47)


2025-05-22 06:03:06.614482: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2052] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 9.0. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.
2025-05-22 06:03:06.637031: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2052] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 9.0. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.
2025-05-22 06:03:08.429181: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78526 MB memory:  -> device: 0, name: NVIDIA H100 80GB HBM3, pci bus id: 0000:1b:00.0, compute capability: 9.0


Epoch 1/50


2025-05-22 06:03:14.605048: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2025-05-22 06:03:16.484110: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f4c15aa6790 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-05-22 06:03:16.484143: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA H100 80GB HBM3, Compute Capability 9.0
2025-05-22 06:03:16.487649: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-05-22 06:03:16.568743: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


36/36 [==============================] - ETA: 0s - loss: 0.9760 - accuracy: 0.4838 - iou_score: 0.0135
Epoch 1: val_iou_score improved from -inf to 0.00782, saving model to Unet_result/models/best_model_foldFold1.h5


...

1/1 [==============================] - 0s 26ms/step
測試結果 - Fold3:
  平均 IoU: 0.5707
  平均誤差(公分): 0.1525
  誤差在0.5公分內準確率: 93.62%
  誤差在1.0公分內準確率: 97.87%

Processing Fold4...
=== 訓練 Fold4 ===
訓練資料: 找到 285 張圖像, 285 張遮罩
驗證資料: 找到 48 張圖像, 48 張遮罩
測試資料: 找到 48 張圖像, 48 張遮罩
未匹配的圖像檔案: ['../data/Fold4/train/img_1.2.826.0.1.3680043.8.498.10001065121843652267743449160233082683.jpg', '../data/Fold4/train/img_1.2.826.0.1.3680043.8.498.10001175380298620851477409998730672515.jpg', '../data/Fold4/train/img_1.2.826.0.1.3680043.8.498.10001274045312501651093242392099983211.jpg', '../data/Fold4/train/img_1.2.826.0.1.3680043.8.498.10002446304107330308555550280339793610.jpg', '../data/Fold4/train/img_1.2.826.0.1.3680043.8.498.10003638361010097105432298560780077394.jpg'] (總共 144 個)
匹配到 141 對圖像和遮罩 (原始圖像數: 285, 原始遮罩數: 285)
匹配到 48 對圖像和遮罩 (原始圖像數: 48, 原始遮罩數: 48)
匹配到 48 對圖像和遮罩 (原始圖像數: 48, 原始遮罩數: 48)
Epoch 1/50
36/36 [==============================] - ETA: 0s - loss: 0.9774 - accuracy: 0.4889 - iou_score: 0.0133
Epoch 1: v

/root/.local/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


36/36 [==============================] - 13s 182ms/step - loss: 0.9774 - accuracy: 0.4889 - iou_score: 0.0133 - val_loss: 0.9836 - val_accuracy: 0.3717 - val_iou_score: 0.0099
Epoch 2/50
36/36 [==============================] - ETA: 0s - loss: 0.9636 - accuracy: 0.6626 - iou_score: 0.0255
Epoch 2: val_iou_score did not improve from 0.00994
36/36 [==============================] - 5s 136ms/step - loss: 0.9636 - accuracy: 0.6626 - iou_score: 0.0255 - val_loss: 0.9843 - val_accuracy: 0.9896 - val_iou_score: 5.9780e-05
Epoch 3/50
36/36 [==============================] - ETA: 0s - loss: 0.9523 - accuracy: 0.8056 - iou_score: 0.0438
Epoch 3: val_iou_score did not improve from 0.00994
36/36 [==============================] - 5s 136ms/step - loss: 0.9523 - accuracy: 0.8056 - iou_score: 0.0438 - val_loss: 0.9842 - val_accuracy: 0.9485 - val_iou_score: 0.0037
Epoch 4/50
36/36 [==============================] - ETA: 0s - loss: 0.9461 - accuracy: 0.8292 - iou_score: 0.0479
Epoch 4: val_iou_score i

/root/.local/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


36/36 [==============================] - 13s 175ms/step - loss: 0.9721 - accuracy: 0.5554 - iou_score: 0.0166 - val_loss: 0.9863 - val_accuracy: 0.9080 - val_iou_score: 2.3414e-04
Epoch 2/50
36/36 [==============================] - ETA: 0s - loss: 0.9543 - accuracy: 0.7586 - iou_score: 0.0365
Epoch 2: val_iou_score improved from 0.00023 to 0.00327, saving model to Unet_result/models/best_model_foldFold5.h5
36/36 [==============================] - 6s 168ms/step - loss: 0.9543 - accuracy: 0.7586 - iou_score: 0.0365 - val_loss: 0.9891 - val_accuracy: 0.6189 - val_iou_score: 0.0033
Epoch 3/50
36/36 [==============================] - ETA: 0s - loss: 0.9417 - accuracy: 0.8372 - iou_score: 0.0526
Epoch 3: val_iou_score improved from 0.00327 to 0.00590, saving model to Unet_result/models/best_model_foldFold5.h5
36/36 [==============================] - 6s 165ms/step - loss: 0.9417 - accuracy: 0.8372 - iou_score: 0.0526 - val_loss: 0.9856 - val_accuracy: 0.9873 - val_iou_score: 0.0059
Epoch 4/50